<a href="https://colab.research.google.com/github/sudip234-source/PyTorch-Tutorial/blob/main/training_pipeline_using_nn_module_%26_optim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [220]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [221]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [222]:
df.shape

(569, 33)

In [223]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [224]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [225]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [226]:
X_train

array([[-1.25408402, -0.28992695, -1.26127953, ..., -1.35958525,
         0.22566259, -0.28422749],
       [-1.02743332, -1.02918209, -1.00106982, ..., -0.32888828,
         0.16824768,  0.67905484],
       [-0.16772373, -1.2850781 , -0.1471935 , ..., -0.36218539,
         0.23807662, -0.11996365],
       ...,
       [-1.23956945, -0.26149406, -1.27463399, ..., -1.73796152,
        -1.55575132, -1.00002622],
       [-0.19563638, -0.13591546, -0.19373178, ...,  0.42181024,
         0.94102131,  0.14546931],
       [ 1.68567615,  1.77856579,  1.67832124, ...,  1.51910143,
         0.557738  , -0.41640005]])

In [227]:
y_test[:2]

,diagnosis
246,B
441,M


In [228]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [229]:
y_test[:2]

array([0, 1])

In [230]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [231]:
X_train_tensor.shape,y_test_tensor.shape

(torch.Size([455, 30]), torch.Size([114]))

## Defining The Model

In [232]:
import torch.nn as nn

In [233]:
class mySimpleNN(nn.Module):
  def __init__(self, num_features):
    super().__init__()

    self.linear = nn.Linear(num_features,16)
    self.sigmoid = nn.Sigmoid()
    self.linear2 = nn.Linear(16,8)
    self.sigmoid2 = nn.Sigmoid()
    self.linear3 = nn.Linear(8,1)
    self.sigmoid3 = nn.Sigmoid()

  def forward(self, features):
    z1 = self.linear(features)
    o1 = self.sigmoid(z1)
    z2 = self.linear2(o1)
    o2 = self.sigmoid2(z2)
    z3 = self.linear3(o2)
    o3 = self.sigmoid3(z3)
    return o3

  def calc_loss(self, y_pred, y):
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
    loss = -(y * torch.log(y_pred) + (1 - y) * torch.log(1 - y_pred)).mean()
    return loss


## Training pipeline

In [234]:
from re import X
X_train_tensor = X_train_tensor.to(torch.float32)
y_train_tensor = y_train_tensor.to(torch.float32)
X_test_tensor = X_test_tensor.to(torch.float32)
y_test_tensor = y_test_tensor.to(torch.float32)

In [235]:
epochs = 500
learning_rate = 0.01

In [236]:
loss_func = nn.BCELoss()
type(loss_func)

torch.nn.modules.loss.BCELoss

In [237]:
model = mySimpleNN(X_train_tensor.shape[1])

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    y_pred = model.forward(X_train_tensor)
    loss = loss_func(y_pred, y_train_tensor.view(-1, 1))
    loss.backward()

    # with torch.no_grad():
    #     model.linear.weight -= learning_rate * model.linear.weight.grad
    #     model.linear.bias -= learning_rate * model.linear.bias.grad
    #     model.linear2.weight -= learning_rate * model.linear2.weight.grad
    #     model.linear2.bias -= learning_rate * model.linear2.bias.grad
    #     model.linear3.weight -= learning_rate * model.linear3.weight.grad
    #     model.linear3.bias -= learning_rate * model.linear3.bias.grad

    #     model.linear.weight.grad.zero_()
    #     model.linear.bias.grad.zero_()
    #     model.linear2.weight.grad.zero_()
    #     model.linear2.bias.grad.zero_()

    optimizer.step()
    optimizer.zero_grad()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")



Epoch 1, Loss: 0.6646025776863098
Epoch 2, Loss: 0.6645054221153259
Epoch 3, Loss: 0.6644096374511719
Epoch 4, Loss: 0.6643151044845581
Epoch 5, Loss: 0.664222002029419
Epoch 6, Loss: 0.6641300320625305
Epoch 7, Loss: 0.6640393137931824
Epoch 8, Loss: 0.6639498472213745
Epoch 9, Loss: 0.6638615727424622
Epoch 10, Loss: 0.6637744903564453
Epoch 11, Loss: 0.663688600063324
Epoch 12, Loss: 0.6636039018630981
Epoch 13, Loss: 0.663520336151123
Epoch 14, Loss: 0.6634379029273987
Epoch 15, Loss: 0.6633564829826355
Epoch 16, Loss: 0.6632763743400574
Epoch 17, Loss: 0.6631971001625061
Epoch 18, Loss: 0.6631189584732056
Epoch 19, Loss: 0.663041889667511
Epoch 20, Loss: 0.6629658341407776
Epoch 21, Loss: 0.6628907322883606
Epoch 22, Loss: 0.6628167629241943
Epoch 23, Loss: 0.6627436876296997
Epoch 24, Loss: 0.6626715064048767
Epoch 25, Loss: 0.6626002788543701
Epoch 26, Loss: 0.6625300645828247
Epoch 27, Loss: 0.6624608039855957
Epoch 28, Loss: 0.6623923778533936
Epoch 29, Loss: 0.662324905395507

In [238]:
print(loss_func(y_pred, y_train_tensor.view(-1, 1)))

tensor(0.6534, grad_fn=<BinaryCrossEntropyBackward0>)


In [239]:
for name, param in model.named_parameters():
    print(f"{name}:")
    print(param.data)
    print("-" * 50)


linear.weight:
tensor([[-0.0959, -0.1854,  0.0528,  0.0611,  0.1498,  0.0982, -0.1380,  0.0392,
          0.0333, -0.0984,  0.0199,  0.0287, -0.1871, -0.0540,  0.0985,  0.1321,
         -0.0911, -0.0230,  0.1779, -0.0693,  0.1606,  0.0324, -0.1012, -0.0795,
         -0.0009,  0.0676,  0.0621,  0.1111,  0.0723,  0.1518],
        [ 0.1067, -0.1475,  0.0606, -0.1633, -0.0432, -0.0770,  0.0986, -0.1198,
          0.0441,  0.1596, -0.0397,  0.1515, -0.0321,  0.0395,  0.0465, -0.0839,
          0.0064,  0.0109, -0.0389, -0.0485,  0.0312,  0.0122, -0.0926,  0.1402,
          0.0211,  0.1251,  0.0525, -0.1508, -0.0608,  0.0714],
        [ 0.0927, -0.1129,  0.0390, -0.0986, -0.0232,  0.0140, -0.0110,  0.1663,
         -0.0739,  0.1637,  0.1338,  0.0396, -0.1449, -0.0419,  0.1476, -0.1298,
          0.1245, -0.0716, -0.1679,  0.0525,  0.0478,  0.0186,  0.1074,  0.0476,
          0.0958, -0.1702,  0.0321, -0.1750, -0.1157, -0.1329],
        [ 0.1776,  0.1085,  0.1227,  0.1075, -0.1367, -0.0728,  

In [240]:
from torchsummary import summary

# Example: suppose your input has 30 features
summary(model, input_size=(30,))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 16]             496
           Sigmoid-2                   [-1, 16]               0
            Linear-3                    [-1, 8]             136
           Sigmoid-4                    [-1, 8]               0
            Linear-5                    [-1, 1]               9
           Sigmoid-6                    [-1, 1]               0
Total params: 641
Trainable params: 641
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [241]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')


Accuracy: 0.6315789222717285
